# Interactive Table Streaming Demo Using 
# Snowpipe Streaming V2 SDK for Real-Time Data Ingestion

##### Author: Colm Moynihan
##### Date: 3 Feb 2026 - Build in London

### Overview
This notebook demonstrates how to ingest JSON data into Snowflake **Interactive Tables** using the **Snowpipe Streaming V2 SDK**. Interactive Tables are optimized for real-time analytics workloads with sub-second query latency.

### What You'll Learn

| Step | Description |
|------|-------------|
| 1️⃣ | Create database, schema, and Interactive Table |
| 2️⃣ | Generate realistic UK web analytics data as JSON |
| 3️⃣ | Stream JSON data using Snowpipe Streaming V2 |
| 4️⃣ | Query and analyze the ingested data |

### Why Snowpipe Streaming V2?

Interactive Tables have a **critical limitation**: they cannot be populated using standard SQL DML statements.

| Method | Works with Interactive Tables? |
|--------|-------------------------------|
| `INSERT INTO` | ❌ No |
| `COPY INTO` | ❌ No |
| Regular Snowpipe | ❌ No |
| **Snowpipe Streaming V2** | ✅ Yes |
| Kafka Connector | ✅ Yes |
| Spark Connector (streaming) | ✅ Yes |

### Prerequisites

```bash
pip install snowpipe-streaming
```

You also need **key-pair authentication** configured for your Snowflake user.

---

## 🔌 Connect to Snowflake

First, establish a connection to Snowflake using the `colms_uswest` connection profile with key-pair authentication.

In [1]:
# Cell: connect_to_snowflake
# Establish connection to Snowflake using colms_uswest profile

import os
import snowflake.connector
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
from tabulate import tabulate

# ============================================
# CONNECTION CONFIGURATION (colms_uswest)
# ============================================
SNOWFLAKE_ACCOUNT = "SFSEEUROPE-COLM_USWEST"
SNOWFLAKE_USER = "ADMIN"
PRIVATE_KEY_PATH = os.path.expanduser("~/.ssh/snowflake/rsa_key.p8")
DATABASE = "DASH_DB"
SCHEMA = "DASH_SCHEMA"
WAREHOUSE = "COMPUTE_WH"
ROLE = "ACCOUNTADMIN"

# ============================================
# Load Private Key
# ============================================
def load_private_key(path):
    with open(path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend()
        )
    return private_key.private_bytes(
        encoding=serialization.Encoding.DER,
        format=serialization.PrivateFormat.PKCS8,
        encryption_algorithm=serialization.NoEncryption()
    )

# ============================================
# Create Connection
# ============================================
print("🔌 Connecting to Snowflake (colms_uswest)...")
print(f"   Account:   {SNOWFLAKE_ACCOUNT}")
print(f"   User:      {SNOWFLAKE_USER}")
print(f"   Role:      {ROLE}")
print(f"   Warehouse: {WAREHOUSE}")

conn = snowflake.connector.connect(
    account=SNOWFLAKE_ACCOUNT,
    user=SNOWFLAKE_USER,
    private_key=load_private_key(PRIVATE_KEY_PATH),
    warehouse=WAREHOUSE,
    role=ROLE,
)

# ============================================
# Helper function to run SQL queries
# ============================================
def run_sql(sql, fetch=True):
    """Execute SQL and optionally return results as a formatted table."""
    cursor = conn.cursor()
    try:
        # Handle multi-statement SQL
        statements = [s.strip() for s in sql.split(';') if s.strip()]
        results = []
        for stmt in statements:
            cursor.execute(stmt)
            if fetch and cursor.description:
                columns = [col[0] for col in cursor.description]
                rows = cursor.fetchall()
                if rows:
                    print(tabulate(rows, headers=columns, tablefmt="pretty"))
                    results.append((columns, rows))
            else:
                print(f"✅ {stmt[:50]}..." if len(stmt) > 50 else f"✅ {stmt}")
        return results
    finally:
        cursor.close()

print("\n✅ Connected to Snowflake successfully!")
print("\n📝 Use run_sql('YOUR SQL HERE') to execute queries.")

🔌 Connecting to Snowflake (colms_uswest)...
   Account:   SFSEEUROPE-COLM_USWEST
   User:      ADMIN
   Role:      ACCOUNTADMIN
   Warehouse: COMPUTE_WH

✅ Connected to Snowflake successfully!

📝 Use run_sql('YOUR SQL HERE') to execute queries.


## 1️⃣ Setup Database & Schema

First, we create a dedicated database and schema to organize our Interactive Table and streaming infrastructure.

In [2]:
# Cell: setup_database
# Create the database and schema
run_sql("""
CREATE DATABASE IF NOT EXISTS INTERACTIVE_JSON_DB;
USE DATABASE INTERACTIVE_JSON_DB;
CREATE SCHEMA IF NOT EXISTS STREAMING;
USE SCHEMA STREAMING
""")

+----------------------------------------------------------+
|                          status                          |
+----------------------------------------------------------+
| INTERACTIVE_JSON_DB already exists, statement succeeded. |
+----------------------------------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+
+------------------------------------------------+
|                     status                     |
+------------------------------------------------+
| STREAMING already exists, statement succeeded. |
+------------------------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+


[(['status'], [('INTERACTIVE_JSON_DB already exists, statement succeeded.',)]),
 (['status'], [('Statement executed successfully.',)]),
 (['status'], [('STREAMING already exists, statement succeeded.',)]),
 (['status'], [('Statement executed successfully.',)])]

## 2️⃣ Create Interactive Table

**Interactive Tables** are a Snowflake table type designed for real-time analytics:

| Feature | Description |
|---------|-------------|
| **Query Latency** | Sub-second response times |
| **Data Freshness** | Near real-time data availability |
| **Clustering** | Automatic micro-partitioning for fast lookups |
| **Use Case** | Dashboards, operational analytics, real-time reporting |

### Table Schema

Our `CUSTOMERS` table stores UK web analytics events with the following columns:

| Column | Type | Description |
|--------|------|-------------|
| `EVENTDATE` | DATE | Date of the web visit |
| `COUNTERID` | NUMBER | Unique session counter |
| `CLIENTIP` | VARCHAR | Visitor's IP address (UK-based) |
| `SEARCHENGINEID` | NUMBER | Search engine source (1=Google, 2=Bing, etc.) |
| `SEARCHPHRASE` | VARCHAR | Search query that led to the visit |
| `RESOLUTIONWIDTH` | NUMBER | Screen resolution width |
| `TITLE` | VARCHAR | Page title visited |
| `ISREFRESH` | NUMBER | Whether page was refreshed (0/1) |
| `DONTCOUNTHITS` | NUMBER | Bot/excluded traffic flag (0/1) |

⚠️ **Important:** Interactive Tables can ONLY be populated via Snowpipe Streaming SDK!

In [3]:
# Cell: create_interactive_table
# Create Interactive Table (for Ingest SDK streaming)
run_sql("""
CREATE OR REPLACE INTERACTIVE TABLE CUSTOMERS CLUSTER BY (CLIENTIP) (
    EVENTDATE DATE,
    COUNTERID NUMBER(38,0),
    CLIENTIP VARCHAR(16777216),
    SEARCHENGINEID NUMBER(38,0),
    SEARCHPHRASE VARCHAR(16777216),
    RESOLUTIONWIDTH NUMBER(38,0),
    TITLE VARCHAR(16777216),
    ISREFRESH NUMBER(38,0),
    DONTCOUNTHITS NUMBER(38,0)
)
""")

+---------------------------------------+
|                status                 |
+---------------------------------------+
| Table CUSTOMERS successfully created. |
+---------------------------------------+


[(['status'], [('Table CUSTOMERS successfully created.',)])]

### Grant Permissions

Grant necessary permissions to allow the streaming role to insert data.

In [4]:
# Cell: grant_permissions
# Grant necessary permissions for streaming
run_sql("""
GRANT USAGE ON DATABASE INTERACTIVE_JSON_DB TO ROLE ACCOUNTADMIN;
GRANT USAGE ON SCHEMA INTERACTIVE_JSON_DB.STREAMING TO ROLE ACCOUNTADMIN;
GRANT INSERT ON TABLE INTERACTIVE_JSON_DB.STREAMING.CUSTOMERS TO ROLE ACCOUNTADMIN
""")

+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+


[(['status'], [('Statement executed successfully.',)]),
 (['status'], [('Statement executed successfully.',)]),
 (['status'], [('Statement executed successfully.',)])]

## 3️⃣ Generate JSON Customer Data

This Python cell generates **500 realistic UK web analytics events** as JSON records. The data simulates:

| Data Element | Description |
|--------------|-------------|
| **UK IP Addresses** | IP prefixes mapped to 10 major UK cities |
| **Search Phrases** | Common UK search queries (NHS, trains, property, etc.) |
| **Page Titles** | Popular UK websites (BBC, Guardian, Amazon.co.uk, etc.) |
| **Screen Resolutions** | Common desktop/mobile resolutions |
| **Date Range** | Events spread across the last 30 days |

The output is a list of **JSON dictionaries** ready for streaming ingestion.

In [5]:
# Cell: generate_uk_data
import json
import random
from datetime import datetime, timedelta

# UK-specific data
UK_CITIES_IPS = {
    "London": ["185.86.", "194.168.", "212.58.", "31.52."],
    "Manchester": ["81.105.", "82.132.", "109.170.", "86.156."],
    "Birmingham": ["92.233.", "86.149.", "109.157.", "81.103."],
    "Leeds": ["90.216.", "86.146.", "109.154.", "81.108."],
    "Glasgow": ["92.238.", "86.159.", "109.148.", "81.111."],
    "Liverpool": ["81.100.", "86.147.", "109.171.", "90.218."],
    "Bristol": ["86.153.", "109.155.", "81.106.", "92.234."],
    "Edinburgh": ["86.158.", "109.149.", "81.112.", "92.239."],
    "Cardiff": ["86.154.", "109.156.", "81.107.", "92.235."],
    "Belfast": ["86.160.", "109.150.", "81.113.", "92.240."],
}

UK_SEARCH_PHRASES = [
    "best fish and chips near me", "weather forecast london", "premier league results",
    "train times to manchester", "nhs appointment booking", "uk visa requirements",
    "cheap flights heathrow", "rightmove houses for sale", "bbc news live",
    "tesco delivery slots", "argos click and collect", "amazon uk prime",
    "royal mail tracking", "dvla tax check", "energy price cap uk",
]

UK_PAGE_TITLES = [
    "BBC News - Home", "The Guardian | News", "Daily Mail Online",
    "Sky News - Breaking News", "Rightmove - UK Property", "Amazon.co.uk",
    "Tesco Groceries Online", "Sainsbury's Shopping", "Argos | Same Day Delivery",
    "John Lewis & Partners", "NHS Health A-Z", "GOV.UK Services",
]

RESOLUTIONS = [1920, 1366, 1536, 1440, 1280, 2560, 3840, 1680]

def generate_uk_ip():
    city = random.choice(list(UK_CITIES_IPS.keys()))
    prefix = random.choice(UK_CITIES_IPS[city])
    return f"{prefix}{random.randint(1, 254)}.{random.randint(1, 254)}"

def generate_customer_event(event_date):
    return {
        "EVENTDATE": event_date.strftime("%Y-%m-%d"),
        "COUNTERID": random.randint(100000, 999999),
        "CLIENTIP": generate_uk_ip(),
        "SEARCHENGINEID": random.choices([1, 2, 3, 4, 5, 0], weights=[50, 20, 10, 10, 5, 5])[0],
        "SEARCHPHRASE": random.choice(UK_SEARCH_PHRASES) if random.random() > 0.3 else "",
        "RESOLUTIONWIDTH": random.choice(RESOLUTIONS),
        "TITLE": random.choice(UK_PAGE_TITLES),
        "ISREFRESH": random.choices([0, 1], weights=[85, 15])[0],
        "DONTCOUNTHITS": random.choices([0, 1], weights=[95, 5])[0],
    }

# Generate 1,000 records
records = []
base_date = datetime.now()
for _ in range(1000):
    event_date = base_date - timedelta(days=random.randint(0, 30))
    records.append(generate_customer_event(event_date))

print(f"✅ Generated {len(records)} UK customer records")
print("\nSample record:")
print(json.dumps(records[0], indent=2))

✅ Generated 1000 UK customer records

Sample record:
{
  "EVENTDATE": "2026-01-05",
  "COUNTERID": 509272,
  "CLIENTIP": "90.218.115.177",
  "SEARCHENGINEID": 4,
  "SEARCHPHRASE": "",
  "RESOLUTIONWIDTH": 2560,
  "TITLE": "John Lewis & Partners",
  "ISREFRESH": 0,
  "DONTCOUNTHITS": 0
}


## 4️⃣ Stream JSON Data via Snowpipe Streaming V2

### How Snowpipe Streaming V2 Works

Snowpipe Streaming V2 provides **low-latency data ingestion** directly into Snowflake tables:

```
┌─────────────────┐     ┌──────────────────┐     ┌─────────────────────┐
│  JSON Records   │ ──► │  Streaming SDK   │ ──► │  Interactive Table  │
│  (Python list)  │     │  (append_row)    │     │  (CUSTOMERS)        │
└─────────────────┘     └──────────────────┘     └─────────────────────┘
```

### Key Components

| Component | Description |
|-----------|-------------|
| **StreamingIngestClient** | Main client that manages the connection to Snowflake |
| **Channel** | A logical stream for writing data to a specific table |
| **Default Pipe** | Auto-generated pipe with naming convention `TABLE_NAME-STREAMING` |
| **append_row()** | Method to insert individual JSON records |

### Authentication

Snowpipe Streaming V2 requires **key-pair authentication**:
- Private key file (`.p8` format)
- Public key registered with your Snowflake user

### Configuration

| Parameter | Value |
|-----------|-------|
| Account | `SFSEEUROPE-COLM_USWEST` |
| User | `ADMIN` |
| Role | `ACCOUNTADMIN` |
| Default Pipe | `CUSTOMERS-STREAMING` |

In [6]:
# Cell: stream_to_interactive_table
# ============================================
# Snowpipe Streaming V2 - Stream data to Interactive Table
# ============================================
# IMPORTANT: This cell uses the Snowpipe Streaming SDK (V2)
# DO NOT use SQL INSERT - it will fail on Interactive Tables!
# ============================================
import os
# ============================================
# CONFIGURATION - Update these values
# ============================================
SNOWFLAKE_ACCOUNT = "SFSEEUROPE-COLM_USWEST"  
SNOWFLAKE_USER = "ADMIN"
PRIVATE_KEY_PATH = os.path.expanduser("~/.ssh/snowflake/rsa_key.p8")

# Target table details
DATABASE = "INTERACTIVE_JSON_DB"
SCHEMA = "STREAMING"
TABLE = "CUSTOMERS"
ROLE = "ACCOUNTADMIN"

# Default pipe naming convention for Interactive Tables: TABLE_NAME-STREAMING
PIPE_NAME = f"{TABLE}-STREAMING"

# ============================================
# Load Private Key as PEM string
# ============================================
def load_private_key_pem(path):
    with open(path, "rb") as key_file:
        return key_file.read().decode('utf-8')

# ============================================
# Stream Data using Snowpipe Streaming V2 SDK
# ============================================
print("=" * 60)
print("SNOWPIPE STREAMING V2 - Interactive Table Data Ingestion")
print("=" * 60)

# Import the Snowpipe Streaming SDK
from snowflake.ingest.streaming import StreamingIngestClient

# Load private key as PEM string
private_key_pem = load_private_key_pem(PRIVATE_KEY_PATH)

print(f"🔌 Connecting to Snowflake...")
print(f"   Account: {SNOWFLAKE_ACCOUNT}")
print(f"   User: {SNOWFLAKE_USER}")
print(f"   Target: {DATABASE}.{SCHEMA}.{TABLE}")
print(f"   Default Pipe: {PIPE_NAME}")

# Create Snowpipe Streaming V2 client
client = StreamingIngestClient(
    client_name=f"{TABLE}_client",
    db_name=DATABASE,
    schema_name=SCHEMA,
    pipe_name=PIPE_NAME,
    properties={
        "account": SNOWFLAKE_ACCOUNT,
        "user": SNOWFLAKE_USER,
        "private_key": private_key_pem,
        "role": ROLE,
        "url": f"https://{SNOWFLAKE_ACCOUNT}.snowflakecomputing.com"
    }
)

# Open a streaming channel to the Interactive Table
print(f"\n📡 Opening streaming channel...")
channel, status = client.open_channel(f"{TABLE}_channel")
print(f"   Channel status: {status}")

# Stream records using Snowpipe Streaming V2 (NOT SQL INSERT!)
print(f"\n🚀 Streaming {len(records)} records via Snowpipe Streaming V2...")
print("   (Using channel.append_row() - NOT SQL INSERT)")

for i, record in enumerate(records):
    # Use Snowpipe Streaming V2 append_row method
    # This is the ONLY way to insert into Interactive Tables
    channel.append_row(record)
    
    # Progress indicator every 100 records
    if (i + 1) % 100 == 0:
        print(f"   ✓ Streamed {i + 1}/{len(records)} records...")

# Close channel and client properly
print("\n📤 Flushing and closing channel...")
channel.close()
client.close()

print(f"\n" + "=" * 60)
print(f"✅ SUCCESS: Streamed {len(records)} records to Interactive Table")
print(f"   Table: {DATABASE}.{SCHEMA}.{TABLE}")
print(f"   Method: Snowpipe Streaming V2 (append_row)")
print("=" * 60)

SNOWPIPE STREAMING V2 - Interactive Table Data Ingestion
2026-01-22T16:50:59Z | INFO | core (246) | thread_unknown | Logger initialized with level: INFO
2026-01-22T16:50:59Z | INFO | core::util::process_info_provider (130) | thread_unknown | Running in Kubernetes pod: false
2026-01-22T16:50:59Z | INFO | core::util::process_info_provider (59) | thread_unknown | Initialized process_info_provider with refresh duration: 50 ms
2026-01-22T16:50:59Z | INFO | core (201) | thread_unknown | SDK execution environment initialized: language=python, version=1.1.2, shaded=false, sdk_id=b98c1b11-034d-4b05-9b0d-168d5122bac1
2026-01-22T16:50:59Z | INFO | core::prod::metrics (87) | thread_unknown | Telemetry exporter initialized without Prometheus recorder
🔌 Connecting to Snowflake...
   Account: SFSEEUROPE-COLM_USWEST
   User: ADMIN
   Target: INTERACTIVE_JSON_DB.STREAMING.CUSTOMERS
   Default Pipe: CUSTOMERS-STREAMING
2026-01-22T16:50:59Z | INFO | core::prod::security::jwt (157) | thread_unknown | Crea

## 5️⃣ Verify Data & Run Analytics

Now that the data has been streamed into the Interactive Table, let's verify the ingestion and run some analytics queries.

### Verification Queries

The following queries demonstrate the **sub-second query latency** of Interactive Tables:

In [7]:
# Cell: setup_interactive_warehouse
# Create and configure Interactive Warehouse for sub-second queries
print("="*60)
print("⚡ SETTING UP INTERACTIVE WAREHOUSE")
print("="*60)

# Create Interactive Warehouse
run_sql("CREATE OR REPLACE INTERACTIVE WAREHOUSE INTERACTIVE_WH_IOT WAREHOUSE_SIZE = 'XSMALL'")

# Add the Interactive Table to the warehouse
run_sql("ALTER WAREHOUSE INTERACTIVE_WH_IOT ADD TABLES (INTERACTIVE_JSON_DB.STREAMING.CUSTOMERS)")

# Resume the warehouse
run_sql("ALTER WAREHOUSE INTERACTIVE_WH_IOT RESUME IF SUSPENDED")

# Use the Interactive Warehouse
run_sql("USE WAREHOUSE INTERACTIVE_WH_IOT")

print()
print("✅ Interactive Warehouse ready for sub-second queries!")

⚡ SETTING UP INTERACTIVE WAREHOUSE
+----------------------------------------------------------------+
|                             status                             |
+----------------------------------------------------------------+
| INTERACTIVE WAREHOUSE INTERACTIVE_WH_IOT successfully created. |
+----------------------------------------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+
+----------------------------------+
|              status              |
+----------------------------------+
| Statement executed successfully. |
+----------------------------------+

✅ Interactive Warehouse ready for sub-second queries!


## 6️⃣ View Streaming Ingestion History & Logs

The Snowpipe Streaming SDK generates logs both **client-side** (in the notebook output) and **server-side** (in Snowflake system views).

### Client-Side Logs
The SDK logs appear in the cell output with entries like:
- `core::channel::flusher` - Channel flush operations
- `core::util::process_info_provider` - Memory/CPU telemetry  
- `core::prod::security::scoped` - JWT token management
- `core::prod::ingest_client` - Client connection events

### Server-Side Logs (ACCOUNT_USAGE Views)
Snowflake tracks streaming ingestion in these system views (note: **~2 hour data latency**):

| View | Purpose |
|------|---------|
| `SNOWPIPE_STREAMING_CHANNEL_HISTORY` | Channel-level metrics: rows inserted, parsed, errors |
| `SNOWPIPE_STREAMING_CLIENT_HISTORY` | Client-level events: blob sizes, event types |
| `PIPE_USAGE_HISTORY` | Pipe usage: credits, bytes inserted/billed |
| `PIPES` | Pipe definitions and metadata |

### Real-Time Monitoring
- `SHOW PIPES` - View active streaming pipes (immediate)

In [ ]:
# Cell: verify_data_count
# ============================================
# Verify Data in Interactive Table (Real-time)
# ============================================
print("=" * 60)
print("✅ DATA VERIFICATION (Real-time)")
print("=" * 60)

run_sql("""
SELECT 
    COUNT(*) AS TOTAL_RECORDS,
    MIN(EVENTDATE) AS EARLIEST_EVENT,
    MAX(EVENTDATE) AS LATEST_EVENT
FROM INTERACTIVE_JSON_DB.STREAMING.CUSTOMERS
""")

✅ DATA VERIFICATION (Real-time)
+---------------+----------------+--------------+
| TOTAL_RECORDS | EARLIEST_EVENT | LATEST_EVENT |
+---------------+----------------+--------------+
|       0       |                |              |
+---------------+----------------+--------------+


[(['TOTAL_RECORDS', 'EARLIEST_EVENT', 'LATEST_EVENT'], [(0, None, None)])]

2026-01-22T16:51:59Z | INFO | core::util::process_info_provider (174) | sf | Current process info: ProcessInfo { process_used_memory_bytes: 154075136, process_total_memory_bytes: 421556584448, pod_used_memory_bytes: 0, pod_total_memory_bytes: 0, system_used_memory_bytes: 35561947136, system_total_memory_bytes: 68719476736, cpu_usage_percent: 0.2850397, num_workers: 10, num_tasks: 2, queue_depth: 0 }
2026-01-22T16:52:59Z | INFO | core::util::process_info_provider (174) | sf | Current process info: ProcessInfo { process_used_memory_bytes: 154075136, process_total_memory_bytes: 421556584448, pod_used_memory_bytes: 0, pod_total_memory_bytes: 0, system_used_memory_bytes: 35607085056, system_total_memory_bytes: 68719476736, cpu_usage_percent: 0.22700243, num_workers: 10, num_tasks: 2, queue_depth: 0 }
2026-01-22T16:53:59Z | INFO | core::util::process_info_provider (174) | sf | Current process info: ProcessInfo { process_used_memory_bytes: 154075136, process_total_memory_bytes: 421556584448, 

---

## 📋 Summary

This notebook demonstrates streaming data directly into an **Interactive Table** using the Snowpipe Streaming SDK.

| Object | Name | Purpose |
|--------|------|---------|
| Database | `INTERACTIVE_JSON_DB` | Container for streaming demo |
| Interactive Table | `CUSTOMERS` | Real-time analytics with sub-second latency |
| Default Pipe | `CUSTOMERS-STREAMING` | System-generated pipe for streaming |

### Key Points:

⚠️ **Interactive Tables can ONLY be populated via:**
- Snowpipe Streaming SDK (this notebook)
- Snowflake Kafka Connector
- Snowflake Spark Connector (streaming mode)

❌ **These do NOT work with Interactive Tables:**
- SQL `INSERT` statements
- `COPY INTO` commands
- Regular Snowpipe

# Generate keypair for authentication
### To stream more data:
Simply run the Python cells again with new records - the Streaming SDK will stream them directly to the Interactive Table!

---

**End of Notebook**